In [1]:

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
import os
from matplotlib import pyplot as plt
import librosa
from glob import glob
from scipy.linalg import circulant
from numba import jit

import sys
sys.path.append('..')
import libfmp.b
import libfmp.c2
import libfmp.c3
import libfmp.c4
from libfmp.c5 import get_chord_labels
import libfmp.c6
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from glob import glob

import librosa
import librosa.display
import IPython.display as ipd

from itertools import cycle

plt.rcParams['figure.figsize'] = (19.2, 10.8) #define size of the plot to 1920*1080 pixs square

import scipy.stats
np.set_printoptions(threshold=sys.maxsize)
import logging

In [2]:
a=libfmp.c5.generate_chord_templates(nonchord=False)
chord_labels = get_chord_labels(nonchord=False)
print(a[:,0])
print(len(chord_labels))

[1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.]
24


In [3]:
path = '/Users/Kittipat/Desktop/fypW/Program/Train_test/Train_songs'  # replace with the path to your directory
files = []

for file_name in os.listdir(path):
    if os.path.isfile(os.path.join(path, file_name)):
        if '.wav' in file_name:
            files.append(file_name)

print(files)

fn_wav = path+'/'+files[0]
N = 4096
H = 2048
X, Fs_X, x, Fs, x_dur = libfmp.c5.compute_chromagram_from_filename(fn_wav, N=N, H=H, gamma=0.1, version='STFT')
chord_sim, chord_max = libfmp.c5.chord_recognition_template(X, norm_sim='max')

fn_wav = path+'/'+files[1]
N = 4096
H = 2048
X2, Fs_X, x, Fs, x_dur = libfmp.c5.compute_chromagram_from_filename(fn_wav, N=N, H=H, gamma=0.1, version='STFT')

['0_Bm_E_C#m_F#m_Bm_E_A_A_110bpm_คนใจง่าย.wav', '0_C#m_B_C#m_B_C#m_B_A_B_112bpm_ขี้หึง.wav', '0_D_D_C#m_C#m_Bm_E_A_A_110bpm_คนใจง่าย.wav', '0_Em7=0.5_Cmaj7=0.5_Em7=0.5_Cmaj7_D_86bpm_ปล่อย.wav', '0_E_E_A_B_C#m_A_B_B_112bpm_ขี้หึง.wav', '0_E_E_G#m_A_E_E_C#m_B_112bpm_ขี้หึง.wav', '0_Fmaj7_Em7_Dm7=2_G7=2_C=2_C7=2_Fmaj7_Em7=2_Am7=2_Dm7_G_68bpm_ถ้า.wav', '0_Fmaj7_Em7_Dm7=2_G7=2_C_Fmaj7_Em7=2_Am7=2_Dm7=2_G7=2_C_68bpm_ถ้า.wav', '0_G=0.5_Em7=0.5_C=0.5_D=0.5_86bpm_ปล่อย.wav', '0_Gmaj7=0.5_Em7=0.5_Cmaj7=0.5_D=0.5_86bpm_ปล่อย.wav', '1_A_Amaj7_D_D_68bpm_นะหน้าธอง.wav', '1_Cmaj7_D7_G_G_100bpm_หมอกจาง.wav', '1_D_Dmaj7_Em7_A_79bpm_ถ้าเธอรักฉันจริง.wav', '1_E_A_C#m_A=2_B=2_73bpm_ร.wav', '1_G_A_F#m7_Bm_110bpm_คุกเข่า.wav', '1_G_A_F#m7_Bm_110bpm_คุกเข่า_stutter.wav', '1_G_Em7_Cmaj7_D_100bpm_หมอกจาง.wav', '2_C_Am7_Dm7_G7_C_Am7_Dm7_G7_68bpm_ถ้า.wav', '2_C_Em7_Am7_F_C_C7_Fmaj7_G_65bpm_ทุ้มอยู่.wav', '2_Dm7_Em7_Fmaj7_Em7_Dm7_C7_Em7_G_G_65bpm_ทุ้มอยู่.wav', '2_G_D_F#_Em7_D_C_Bm7_Am7_D_76bpm_ปีนี้ไม่ต้องเหงา.w

In [4]:
print(chord_sim[0,0])
print(chord_max.shape[1])
chord_labels = get_chord_labels(nonchord=False)
chord_labels.index('D')

0.6059947274899343
188


2

In [5]:
all_sim = np.empty([24,0])
all_real = np.empty([0],dtype=int)
chord_labels = get_chord_labels(nonchord=False)
count = 0
for items in files:
    fn_wav = path+'/'+items
    N = 4096
    H = 2048
    X, Fs_X, x, Fs, x_dur = libfmp.c5.compute_chromagram_from_filename(fn_wav, N=N, H=H, gamma=0.1, version='STFT')
    chord_sim, chord_max = libfmp.c5.chord_recognition_template(X, norm_sim='max')
    all_sim = np.append(all_sim, chord_sim, axis=1)
    df = libfmp.b.b_annotation.read_csv((path+'/label/'+items.split('.wav')[0] + '.csv'), header=True, add_label=False)
    all_real = np.append(all_real, np.zeros((chord_sim.shape[1]),dtype=int))
    #print(all_real)
    for i in range(df.shape[0]):
        df_start = df.at[i,'start']
        df_end = df.at[i,'end']
        first_frame = round(df_start/0.09284)
        last_frame = round(df_end/0.09284)
        chord_each = df.at[i,'label']
        if 'maj7' in chord_each:
            chord = chord_each[0]
        elif 'm7' in chord_each:
            chord = chord_each[0] + chord_each[1]
        elif '7' in chord_each:
            chord = chord_each[0]
        else:
            chord = chord_each
        for n in range(first_frame, last_frame):
            #print(n)
            ind = chord_labels.index(chord)
            if (n+count)<len(all_real):
                all_real[n+count] = ind
                #print(ind)
            #print(all_real[i])
    count += chord_sim.shape[1]-1
    

print(all_sim.shape)
print(all_real.shape)
#print(all_real)

(24, 4798)
(4798,)


In [6]:
a = np.transpose(all_sim)
print(a.shape)
y_train = np.empty([3360], dtype=int)
y_test = np.empty([1438], dtype=int)
for i in range(3360):
    y_train[i] = int(all_real[i])
for i in range(1438):
    y_test[i] = int(all_real[i+3360])
    
#print(y_train)
#print(y_test.shape,y_train.shape,X_train.shape,X_test.shape)

(4798, 24)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(a, all_real, test_size=0.3, random_state=42)
clf = SVC()
clf.fit(X_train, y_train)

# Evaluate the performance of the model on the testing set
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)

Accuracy: 0.6263888888888889
Precision: 0.6204716189953803
Recall: 0.6263888888888889
F1-score: 0.6113714525184756


C:\Users\Kittipat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
print(y_test.shape)

(1440,)


In [8]:
X_pred = clf.predict(a)
accuracy = accuracy_score(all_real, X_pred)
print('Accuracy:', accuracy)

Accuracy: 0.6511046269278866
